fonction qui utilise les embeddings pour trouver les films les plus similaire

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [18]:
# Charger le fichier JSON
movies_df = pd.read_json('cleaned_movies.json', lines=True)

# Afficher un aperçu des données
print(movies_df.head())


                         names       date_x  score                                          genre  \
0                    Creed III  03/02/2023      73                                  Drama, Action   
1     Avatar: The Way of Water  12/15/2022      78             Science Fiction, Adventure, Action   
2  The Super Mario Bros. Movie  04/05/2023      76  Animation, Adventure, Family, Fantasy, Comedy   
3                      Mummies  01/05/2023      70  Animation, Comedy, Family, Adventure, Fantasy   
4                    Supercell  03/17/2023      61                                         Action   

                                            overview                   orig_title     status  \
0  After dominating the boxing world, Adonis Cree...                    Creed III   Released   
1  Set more than a decade after the events of the...     Avatar: The Way of Water   Released   
2  While working underground to fix a water main,...  The Super Mario Bros. Movie   Released   
3  Throug

new


In [7]:
!pip install pandas sentence-transformers openai transformers scikit-learn



In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer



In [9]:
movies_df.head()

,names,date_x,score,genre,overview,orig_title,status,orig_lang,budget_x,revenue,country,keywords,release_year,profit,embeddings
0,Creed III,03/02/2023,73,"Drama, Action","After dominating the boxing world, Adonis Cree...",Creed III,Released,English,75000000.0,2.716167e+08,AU,"[drama, action]",2023,1.966167e+08,"[-0.01651798, 0.08433093, -0.07042654, 0.00923..."
1,Avatar: The Way of Water,12/15/2022,78,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU,"[science fiction, adventure, action]",2022,1.856795e+09,"[-0.10121736, 0.008698871, -0.030430483, 0.055..."
2,The Super Mario Bros. Movie,04/05/2023,76,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU,"[animation, adventure, family, fantasy, comedy]",2023,6.244590e+08,"[-0.010655896, -0.012520119, 0.0050062975, -0...."
3,Mummies,01/05/2023,70,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU,"[animation, comedy, family, adventure, fantasy]",2023,2.190000e+07,"[-0.010687406, 0.008270726, 0.0110955145, -0.0..."
4,Supercell,03/17/2023,61,Action,Good-hearted teenager William always lived in ...,Supercell,Released,English,77000000.0,3.409420e+08,US,[action],2023,2.639420e+08,"[-0.013036367, 0.07371271, 0.014476209, -0.021..."


In [10]:
# Charger le modèle d'embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Générer les embeddings pour les descriptions des films
movies_df['embeddings'] = movies_df['overview'].apply(lambda x: embedding_model.encode(x))

print("Embeddings générés pour chaque description.")


Embeddings générés pour chaque description.


In [11]:
# Fonction pour rechercher des films
def search_movies(query, movies_df, top_n=5):
    # Créer l'embedding pour la requête utilisateur
    query_embedding = embedding_model.encode(query)

    # Calculer la similarité cosinus avec les embeddings des films
    similarities = cosine_similarity([query_embedding], np.vstack(movies_df['embeddings']))

    # Obtenir les indices des films les plus similaires
    top_indices = np.argsort(similarities[0])[-top_n:][::-1]

    # Retourner les films correspondants
    return movies_df.iloc[top_indices][['names', 'overview', 'genre', 'score']]

# Exemple d'utilisation de la recherche
query = "Je veux voir un film d'action et d'aventure."
recommended_movies = search_movies(query, movies_df)
print(recommended_movies)


                                               names  \
9626                                        Euphoria   
4763                                           Mayaa   
1742  X-Rated: The Greatest Adult Movies of All Time   
4212                                  Les Misérables   
4653                              Notre-Dame on Fire   

                                               overview            genre  score  
9626  Artist and filmmaker Julian Rosefeldt creates ...          Unknown      0  
4763                          Raajhorshee De's Macbeth.            Crime      0  
1742  The evolution of adult cinema through the most...      Documentary     63  
4212  An adaptation of the successful stage musical ...   History, Drama     73  
4653  A film relating from the inside the Notre-Dame...  Drama, Thriller     66  


In [12]:
# Charger le modèle GPT et le tokenizer
gpt_model_name = "gpt2"  # Vous pouvez choisir un autre modèle
gpt_model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
gpt_tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_name)
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:
# Fonction pour générer une réponse avec GPT
def generate_response_with_gpt(movies, query):
    # Préparer une liste des recommandations
    recommendations = "\n".join(
        [f"{row['names']} - {row['genre']} (Score: {row['score']})" for _, row in movies.iterrows()]
    )

    # Prompt pour GPT
    prompt = f"""L'utilisateur a demandé : "{query}".
Voici des recommandations de films basées sur sa demande :
{recommendations}

Rédigez une réponse engageante pour l'utilisateur expliquant pourquoi ces films pourraient être intéressants.
"""

    # Encodage du prompt pour GPT
    inputs = gpt_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Générer du texte avec GPT
    outputs = gpt_model.generate(
        inputs.input_ids,
        max_length=200,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=gpt_tokenizer.pad_token_id,
    )

    # Décoder la sortie générée
    generated_text = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraire uniquement la partie générée après le prompt
    return generated_text[len(prompt):].strip()


In [14]:
# Pipeline complet pour la recommandation
def recommend_movies(query, movies_df, top_n=5):
    # Étape 1 : Recherche des films les plus pertinents
    recommended_movies = search_movies(query, movies_df, top_n=top_n)

    # Étape 2 : Génération de réponse avec GPT
    response = generate_response_with_gpt(recommended_movies, query)

    return response, recommended_movies


In [16]:
# Query de l'utilisateur
query = "Je cherche un film de science-fiction avec une histoire captivante."

# Appeler la fonction de recommandation
response, recommended_movies = recommend_movies(query, movies_df)

# Afficher les résultats des recommandations
print("🎬 Recommandations de films :\n")
for index, movie in recommended_movies.iterrows():
    print(f"📽️ **Titre** : {movie['names']}")
    print(f"   🏷️ Genre : {movie['genre']}")
    print(f"   ⭐ Score : {movie['score']}/100")
    print(f"   📝 Description : {movie['overview']}")
    print("-" * 50)

# Afficher la réponse générée par ChatGPT
print("\n🤖 Réponse générée par GPT :")
print(response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🎬 Recommandations de films :

📽️ **Titre** : Notre-Dame on Fire
   🏷️ Genre : Drama, Thriller
   ⭐ Score : 66/100
   📝 Description : A film relating from the inside the Notre-Dame de Paris fire of April 2019.
--------------------------------------------------
📽️ **Titre** : Euphoria
   🏷️ Genre : Unknown
   ⭐ Score : 0/100
   📝 Description : Artist and filmmaker Julian Rosefeldt creates elaborately staged films that investigate the power of language and the conventions of cinema as an allegory for societal and individual behaviors. With the multi-channel film installation Euphoria he continues this examination by exploring capitalism, colonialism, and the influential effects of unlimited economic growth in society.
--------------------------------------------------
📽️ **Titre** : Vico C: The Life of a Philosopher
   🏷️ Genre : Drama
   ⭐ Score : 74/100
   📝 Description : A biographical film based on the story of the singer, songwriter and pioneer of urban music in Spanish. From his hum